In [3]:
%matplotlib inline
from __future__ import division
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
plt.rcParams["animation.ffmpeg_path"] = "C:\FFMPEG"
import matplotlib.animation as manimation
import io

In [4]:
#Random Spin Configuration
def randominitial(N):
    rstate = 2*np.random.randint(2, size = (N, N)) - 1
    return rstate

#Fully Magnetized, all spins aligned
def fullinitial(N):
    fstate = np.ones((N, N), dtype = int)
    return fstate

#Monte Carlo move using Metropolis algorithm
def mcmove(config, beta):
    for i in range(N):
        for j in range(N):
            a = np.random.randint(0, N)
            b = np.random.randint(0, N)
            s = config[a, b]
            nb = config[(a + 1)%N, b] + config[a, (b + 1)%N] + config[(a - 1)%N, b] + config[a, (b - 1)%N]
            cost = 2*s*nb
            if cost < 0:
                s *= -1
            elif rand() < np.exp(-cost*beta):
                s *=-1
            config[a, b] = s
        return config
#Energy of a given configuration
def calcEnergy(config):
    energy = 0
    for i in range(len(config)):
        for j in range(len(config)):
            S = config[i, j]
            nb = config[(i + 1)%N, j] + config[i, (j + 1)%N] + config[(i - 1)%N, j] + config[i, (j - 1)%N]
            energy += -nb*S
    return energy/4.

#Magnetization of a given configuration
def calcMag(config):
    mag = np.sum(config)
    return mag

#change the parameter below if you want to simulate a smaller system
nt = 500 #2**8         #number of temperature points
N = 10 #2**4           #size of the lattice N x N
eqSteps = 150 #2**10   #number of MC sweeps for equilibration
mcSteps = 100 #2**10   #number of MC sweeps for calculation

n1, n2 = 1.0/(mcSteps*N*N), 1.0/(mcSteps*mcSteps*N*N)
tm = 2.269;     T = np.random.normal(tm, .64, nt)
T = T[(T > 1.2) & (T < 3.8)];     nt = np.size(T)

#Empty array for random start values
Energy = np.zeros(nt);     Magnetization = np.zeros(nt)

In [ ]:
initial_state = float(input("Which configuration do you want to run?\n 0: randomly mangetized \n 1: fully magnetized \n"))

if initial_state == 0:
     config = randominitial(N)

elif initial_state == 1:
    config = fullinitial(N)
    
for m in range(len(T)):
    E1 = M1 = E2 = M2 = 0
    iT = 1.0/T[m]; iT2 = iT*iT
       
    for i in range(eqSteps):     #equilibrate
        mcmove(config, iT)       #Monte Carlo moves
        
    for i in range(mcSteps):
        mcmove(config, iT)
        Ene = calcEnergy(config)  #calculate energy
        Mag = calcMag(config)     #calculate the magnetisation
        
        E1 = E1 + Ene
        M1 = M1 + Mag
        M2 = M2 + Mag*Mag
        E2 = E2 + Ene*Ene
        
        Energy[m] = n1*E1
        Magnetization[m] = n1*M1
        #SpecificHeat[m] = (n1*E2 - n2*E1*E1)*iT2
        #Susceptibility[m] = (n1*M2 - n2*M1*M1)*iT
print(config)
#Graph
f = plt.figure(figsize = (18, 10)); #plot calculated values
sp = f.add_subplot(2, 2, 2);
plt.plot(T, Energy, '.', color = 'forestgreen');
plt.xlabel("Temperature (T)", fontsize = 20);
plt.ylabel("Energy", fontsize = 20);
        
sp = f.add_subplot(2, 2, 1);
plt.plot(T, abs(Magnetization), '.', color = 'steelblue');
plt.xlabel("Temperature (T)", fontsize = 20);
plt.ylabel("Energy", fontsize = 20);

if initial_state == 0:
    plt.suptitle("For a randomly magnetized lattice", fontsize = 30)
elif initial_state == 1:
    plt.suptitle("For a fully magnetized lattice", fontsize = 30)

In [104]:
class Ising:
    
    _steps = int(2e6)
    
    def __init__(self, temp, initial_state, size = (100, 100), show = False):
        self.T = temp
        self.sqr_size = size[0]
        self.initialize(initial_state)
        
    def initialize(self, initial_state):
        if initial_state == 0:
            system = randominitial(N)
            system[system == 0] = -1
        elif initial_state == 1:
            system = fullinitial(N)
            system[system == 0] = -1
        self.system = system
    
    #periodic boundary condition heck if a lattice site coordinate falls out of bounds. If it does,
    #apply periodic boundary condition 
    def periodic_bound(self, i):
        if i + 1 > self.size - 1:
            return 0
        elif i - 1 < 0:
            return self.size
        else:
            return i
    def _energy(self, x, y):
        #Calc energy of spin interaction at one lattice point (x,y) with four nearest neighbors
        return -2*self.system[x, y]*(
                    self.system[self.periodic_bound(x - 1), y] 
                    + self.system[self.periodic_bound(x + 1), y] 
                    + self.system[x, self.periodic_bound(y - 1)] 
                    + self.system[x, self.periodic_bound(y + 1)]
                    )
    @property #backward compatible
    def internal_energy(self):
        e = 0; E = 0; E_2 = 0

        for i in range(self.size):
            for j in range(self.size):   
                e = self._energy(i, j)
                E += e
                E_2 += e**2

        U = (1./self.size**2)*E
        U_2 = (1./self.size**2)*E_2 

        return U, U_2
    
    #Find the overall magnetization of the system
    @property
    def magnetization(self):
        return np.abs(np.sum(self.system)/self.size**2)
    
    #Run the simulation
    def simulate(self, video = True):
        FFMpegWriter = manimation.writers['FFMPEG']
        writer = FFMpegWriter(fps = 10)
    

        plt.ion()
        fig = plt.figure()

        with writer.saving(fig, "ising.mp4", 100):
            for step in range(self._steps):
                #pick random lattice point
                x, y = np.random.randint(0, self.size, 2)

                #calc energy of a flipped spin
                E = -1*self._energy(x, y)

                print(E)
                #conditions for whether or not spin will flip (preferred state)
                if E <= 0.:
                    self.system[x, y]*=-1
                elif np.exp(-E/self.T) > np.random.random():
                    self.system[x, y]*=-1

                if step % (self._steps//75) == 0: 
                    print("Step {:.2E} / {:.2E}".format(step, self._steps), end ='\t')
                    print("Percent completed {:.2f}%".format(100*step/self._steps), end ='\t')
                    print("Net Magnetization: {:.2f}".format(self.magnetization))
                    if video:
                        img = plt.imshow(self.system, interpolation = 'nearest')
                        writer.grab_frame()
                        img.remove()
                
            print("...done")

        plt.close('all')
        
lattice = Ising(temp = .5, initial_state = initial_state, size=(100, 100))
lattice.simulate()
    

RuntimeError: No MovieWriters available!